# Converting Kloppy Dataset to Polars DataFrame

### Setup
Start by loading some event data using the Kloppy module. For the sake of this demonstration, we will use Statsbomb Open Event Data.

In [ ]:
from kloppy import statsbomb

# Load Statsbomb open dataset
dataset = statsbomb.load_open_data(
    match_id=15946,
    # Optional arguments
    coordinates="statsbomb",
)

Test if the loading worked by printing the home and away teams.

In [2]:
# Get teams
home_team, away_team = dataset.metadata.teams
f"{home_team} vs {away_team}"

'Barcelona vs Deportivo Alavés'

### Filter the `shot` events from the dataset. 

In [3]:
# Only keep shots
shots = dataset.filter("shot")

### Convert to Polars DataFrame
Parameters:

- `player_id`: Includes the player’s unique identifier.

- Lambda function: Extracts `player_name` and `is_goal` status from each shot event.
    - `player_name`: Converts the player object to a string.
    - `is_goal`: Retrieves whether the shot resulted in a goal (True or False).

- `coordinates_*`: Includes all coordinate-related fields in the dataset.

- `prev_pass_player`: Captures the player who made the pass before the shot.

- `engine="polars"`: Specifies the use of the Polars library for DataFrame processing. Alternatively, using `engine="pandas"` would convert the dataset into a Pandas DataFrame. 

In [5]:
# Convert Kloppy dataset to Polars DataFrame
shots.to_df(
    "player_id",
    lambda event: {
        "player_name": str(event.player),
        "is_goal": event.result.is_success,
    },
    "coordinates_*",
    prev_pass_player=lambda event: str(event.prev("pass").player),
    engine="polars",
)

player_id,player_name,is_goal,coordinates_x,coordinates_y,prev_pass_player
str,str,bool,f64,f64,str
"""5503""","""Lionel Andrés Messi Cuccittini""",false,111.45,52.85,"""Ivan Rakitić"""
"""5211""","""Jordi Alba Ramos""",false,113.85,26.35,"""Lionel Andrés Messi Cuccittini"""
"""5503""","""Lionel Andrés Messi Cuccittini""",false,93.65,34.65,"""Rubén Duarte Sánchez"""
"""6613""","""Rubén Sobrino Pozuelo""",false,109.15,39.05,"""Manuel Alejandro García Sánche…"
"""5246""","""Luis Alberto Suárez Díaz""",false,107.75,24.65,"""Ousmane Dembélé"""
…,…,…,…,…,…
"""6935""","""Adrián Marín Gómez""",false,114.45,32.75,"""Ibai Gómez Pérez"""
"""3501""","""Philippe Coutinho Correia""",false,113.15,31.35,"""Lionel Andrés Messi Cuccittini"""
"""3501""","""Philippe Coutinho Correia""",true,105.25,33.35,"""Arthur Henrique Ramos de Olive…"
